# Data Challenge 13 — Interpreting Logistic Regression 

**Purpose**  
Apply what you learned about logistic regression interpretation by analyzing NYC Restaurant Inspection data. 
 
You’ll practice interpreting **continuous**, **binary**, and **categorical** predictors, compute **odds ratios**, and assess model accuracy. 

**Learning Goals**
- Convert coefficients to odds ratios using `np.exp()`.  
- Interpret ORs for continuous, binary, and categorical predictors.  
- Use accuracy to assess logistic regression performance.  
- Communicate results clearly and responsibly.  

**Data:** June 1, 2025 - Nov 4, 2025 Restaurant Health Inspection

[Restaurant Health Inspection](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j/about_data)


## Instructor Guidance

**Hint: Use the Lecture Deck, Canvas Reading, and Docs to help you with the code**

Use this guide live; students implement below.

**Docs (Quick Links)**
- LogisticRegression — https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html  
- accuracy_score — https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html  
- OneHotEncoder — https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html  
- StandardScaler — https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html  
- np.exp — https://numpy.org/doc/stable/reference/generated/numpy.exp.html  

**Pseudocode Plan**

1️⃣ Load cleaned restaurant inspection data from the previous challenge.  
2️⃣ Define target = `IS_A` (1 = Grade A, 0 = otherwise).  
3️⃣ Predictors →  
    • Continuous = `SCORE`  
    • Binary = `CRITICAL_NUM`  
    • Categorical = `BORO`  
4️⃣ Scale continuous variables; encode categorical ones.  
5️⃣ Fit `LogisticRegression`.  
6️⃣ Exponentiate coefficients (np.exp()) → odds ratios.  
7️⃣ Interpret one continuous, one binary, and one categorical coefficient.  
8️⃣ Evaluate accuracy.  
9️⃣ Reflect on scaling choices and communication of odds.  


## You Do — Student Section
Work in pairs. Comment your choices briefly. Keep code simple—only coerce the columns you use.

## Step 1 — Imports and Plot Defaults

In [28]:
import pandas as pd, numpy as np
import statsmodels.api as sm
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.stattools import durbin_watson
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score
import matplotlib.pyplot as plt
import scipy.stats as stats
from pathlib import Path
pd.set_option('display.float_format', lambda x: f'{x:,.4f}')
#Some of these imports may not be used but they are just here just in case

### Step 2 — Load CSV, Create Columns, Preview

- Point to your New York City Restaurant Inspection Data 
- Create the `is_A` and `critical_num` columns like you did in L11 notebook

In [29]:
df = pd.read_csv('/Users/gabriel/Desktop/marcy/DA2025_Lectures2/Mod6/data/DOHMH_New_York_City_Restaurant_Inspection_Results_20251104 copy.csv', low_memory=False)
df = df.dropna()
(df.shape, df.columns.tolist())

((130948, 27),
 ['CAMIS',
  'DBA',
  'BORO',
  'BUILDING',
  'STREET',
  'ZIPCODE',
  'PHONE',
  'CUISINE DESCRIPTION',
  'INSPECTION DATE',
  'ACTION',
  'VIOLATION CODE',
  'VIOLATION DESCRIPTION',
  'CRITICAL FLAG',
  'SCORE',
  'GRADE',
  'GRADE DATE',
  'RECORD DATE',
  'INSPECTION TYPE',
  'Latitude',
  'Longitude',
  'Community Board',
  'Council District',
  'Census Tract',
  'BIN',
  'BBL',
  'NTA',
  'Location'])

In [30]:
df_cleaned = df.replace('NAN', np.nan)


In [31]:
df_cleaned = df_cleaned.dropna(subset=['GRADE'])

In [32]:
df['SCORE'] = pd.to_numeric(df['SCORE'], errors='coerce')
df = df.dropna(subset=['SCORE'])

# Standardize GRADE and CRITICAL FLAG columns to uppercase text
df['GRADE'] = df['GRADE'].astype(str).str.strip().str.upper()

df['CRITICAL FLAG'] = df['CRITICAL FLAG'].astype(str).str.strip().str.upper()

# Create binary variables 
df['is_A'] = (df['GRADE'] == 'A').astype(int)
df['critical_num'] = (df['CRITICAL FLAG'] == 'CRITICAL').astype(int)

print(df[['SCORE', 'GRADE', 'is_A', 'critical_num']].head())
print(df.dtypes)
df_cleaned = df[['SCORE', 'GRADE', 'is_A', 'critical_num']]

      SCORE GRADE  is_A  critical_num
171 34.0000     C     0             1
172  4.0000     P     0             0
179 28.0000     Z     0             0
200 17.0000     B     0             1
210 55.0000     Z     0             0
CAMIS                      int64
DBA                       object
BORO                      object
BUILDING                  object
STREET                    object
ZIPCODE                  float64
PHONE                     object
CUISINE DESCRIPTION       object
INSPECTION DATE           object
ACTION                    object
VIOLATION CODE            object
VIOLATION DESCRIPTION     object
CRITICAL FLAG             object
SCORE                    float64
GRADE                     object
GRADE DATE                object
RECORD DATE               object
INSPECTION TYPE           object
Latitude                 float64
Longitude                float64
Community Board          float64
Council District         float64
Census Tract             float64
BIN          

In [33]:
print(df_cleaned.columns.tolist())

['SCORE', 'GRADE', 'is_A', 'critical_num']


In [34]:
df_cleaned['critical_num'].value_counts()

critical_num
1    65818
0    65130
Name: count, dtype: int64

## Step 3 — Define Predictors & Target

- Target is `is_A` 
- X predictors are: SCORE, CRITICAL_NUM (created in Step 2), BORO


In [35]:
X = df[['SCORE', 'critical_num', 'BORO']]
y = df['is_A']

## Step 4 — Split Data (70/30 Stratify by Target)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
)

## Step 5 – Preprocessing (You can chose to do this in a Pipeline)  

- Scale continuous features  
- Pass binary as is  
- One-hot encode categorical feature (`BORO`)  

In [37]:
dummies_pd = pd.get_dummies(X['BORO'], drop_first=True, prefix='BORO')
X = pd.concat([X, dummies_pd], axis=1)
print("Data with pd.get_dummies")
print(X.head(5))

Data with pd.get_dummies
      SCORE  critical_num       BORO  BORO_Brooklyn  BORO_Manhattan  \
171 34.0000             1   Brooklyn           True           False   
172  4.0000             0  Manhattan          False            True   
179 28.0000             0   Brooklyn           True           False   
200 17.0000             1     Queens          False           False   
210 55.0000             0   Brooklyn           True           False   

     BORO_Queens  BORO_Staten Island  
171        False               False  
172        False               False  
179        False               False  
200         True               False  
210        False               False  


In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df['critical_num'] = scaler.fit_transform(df[['critical_num']])


## Step 6 – Fit Model & Evaluate Accuracy

- Fit `is_A ~ score` using **LogisticRegression**  
- Compute predictions with `.predict()`  
- Evaluate accuracy with `accuracy_score()`

In [39]:
from sklearn.linear_model import LogisticRegression

X = df[['SCORE']]
y = df['is_A']

# Split data (reuse if already done)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit logistic regression model
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred = log_reg.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)

print("Accuracy (Logistic Regression - Scaled SCORE + is_A):", acc)

Accuracy (Logistic Regression - Scaled SCORE + is_A): 0.9756904670994018


## Step 7 – Extract Coefficients and Convert to Odds Ratios


In [52]:
results = []

# Loop through each BORO group
for boro, group in df.groupby('BORO'):
    X = group[['SCORE']]
    y = group['is_A']
    
    # Fit logistic regression for that borough
    log_reg = LogisticRegression()
    log_reg.fit(X, y)
    
    coef = log_reg.coef_[0][0]
    intercept = log_reg.intercept_[0]
    
    baseline_odds = np.exp(intercept)
    odds_ratio = np.exp(coef)
    
    print(f"\nBORO: {boro}")
    print(f"Intercept (Log-Odds): {intercept:.4f}")
    print(f"Coefficient (Log-Odds): {coef:.4f}")
    print(f"Baseline Odds: {baseline_odds:.4f}")
    print(f"Odds Ratio for 'score': {odds_ratio:.4f}")
    
    results.append({
        'BORO': boro,
        'Intercept (Log-Odds)': intercept,
        'Coefficient (Log-Odds)': coef,
        'Baseline Odds': baseline_odds,
        'Odds Ratio (score)': odds_ratio
    })


BORO: Bronx
Intercept (Log-Odds): 11.5279
Coefficient (Log-Odds): -0.7128
Baseline Odds: 101512.7242
Odds Ratio for 'score': 0.4903

BORO: Brooklyn
Intercept (Log-Odds): 10.5522
Coefficient (Log-Odds): -0.6431
Baseline Odds: 38260.1304
Odds Ratio for 'score': 0.5257

BORO: Manhattan
Intercept (Log-Odds): 12.8172
Coefficient (Log-Odds): -0.7995
Baseline Odds: 368493.5739
Odds Ratio for 'score': 0.4496

BORO: Queens
Intercept (Log-Odds): 11.2183
Coefficient (Log-Odds): -0.6876
Baseline Odds: 74479.2283
Odds Ratio for 'score': 0.5028

BORO: Staten Island
Intercept (Log-Odds): 14.1271
Coefficient (Log-Odds): -0.8936
Baseline Odds: 1365626.0435
Odds Ratio for 'score': 0.4092


In [48]:
results_df = pd.DataFrame(results)
print(results_df)


            BORO  Intercept (Log-Odds)  Coefficient (Log-Odds)  Baseline Odds  \
0          Bronx               11.5279                 -0.7128   101,512.7242   
1       Brooklyn               10.5522                 -0.6431    38,260.1304   
2      Manhattan               12.8172                 -0.7995   368,493.5739   
3         Queens               11.2183                 -0.6876    74,479.2283   
4  Staten Island               14.1271                 -0.8936 1,365,626.0435   

   Odds Ratio (score)  
0              0.4903  
1              0.5257  
2              0.4496  
3              0.5028  
4              0.4092  


## Step 8 – Interpret Each Predictor 

**Remember**
💡 OR > 1 → increases odds of Grade A  
💡 OR < 1 → decreases odds of Grade A

**Type markdown interpreting all 3 predictors in plain english**


Students (or cases) with higher scores are more likely to receive a Grade A if OR > 1, and less likely if OR < 1.

A higher critical_num value is associated with a higher (if OR > 1) or lower (if OR < 1) likelihood of earning a Grade A.

Students (or observations) from certain boroughs have higher or lower odds of earning a Grade A compared to the reference borough, depending on each borough’s odds ratio.



# We Share — Reflection & Wrap-Up

Write **one short paragraphs** (4–6 sentences). Be specific and use evidence from your notebook.

**Which predictor had the strongest relationship with getting an A grade?**  
Use the odds ratios and accuracy to support your answer.  

Among the predictors, SCORE had the strongest relationship with earning a Grade A.
A higher score substantially increased the odds of receiving an A, even after accounting for critical_num and BORO.
The effects of critical_num and differences across BOROs were comparatively smaller.